In [62]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import imageio
import os 


## Preprocessing Images

In [220]:
# ## Train Positive
# pos_train_path = "Data/chest_xray/train/PNEUMONIA/"
# train_pos_images = os.listdir(pos_train_path)
# resized_positive_save_path = "resizedImages/train/PNEUMONIA/"
# for x in train_pos_images:
#     img_path = pos_train_path+x
#     img_path = tf.io.read_file(img_path)
#     img = tf.image.decode_jpeg(img_path, channels=3)
#     resized_img = tf.dtypes.cast(tf.image.resize(img, (128,128)), tf.float32)
#     imageio.imwrite(resized_positive_save_path+x, resized_img)
#     print("img {} done".format(x))    

# print("******************** TRAIN POS COMPLETED *********************************")
# ## Train Negative

# neg_train_path = "Data/chest_xray/train/NORMAL/"
# train_neg_images = os.listdir(neg_train_path)
# resized_negative_save_path = "resizedImages/train/NORMAL/"
# for x in train_neg_images:
#     img_path = neg_train_path+x
#     img_path = tf.io.read_file(img_path)
#     img = tf.image.decode_jpeg(img_path, channels=3)
#     resized_img = tf.dtypes.cast(tf.image.resize(img, (128,128)), tf.float32)
#     imageio.imwrite(resized_negative_save_path+x, resized_img)
#     print("img {} done".format(x))    
# print("******************** TRAIN NEG COMPLETED *********************************")

# ## Test Positive
# pos_test_path = "Data/chest_xray/test/PNEUMONIA/"
# test_pos_images = os.listdir(pos_test_path)
# resized_positive_save_path = "resizedImages/test/PNEUMONIA/"
# for x in test_pos_images:
#     img_path = pos_test_path+x
#     img_path = tf.io.read_file(img_path)
#     img = tf.image.decode_jpeg(img_path, channels=3)
#     resized_img = tf.dtypes.cast(tf.image.resize(img, (128,128)), tf.float32)
#     imageio.imwrite(resized_positive_save_path+x, resized_img)
#     print("img {} done".format(x))    

# print("******************** TEST POS COMPLETED *********************************")

# ## Test Negative
# neg_test_path = "Data/chest_xray/test/NORMAL/"
# test_neg_images = os.listdir(neg_test_path)
# resized_negative_save_path = "resizedImages/test/NORMAL/"
# for x in test_neg_images:
#     img_path = neg_test_path+x
#     img_path = tf.io.read_file(img_path)
#     img = tf.image.decode_jpeg(img_path, channels=3)
#     resized_img = tf.dtypes.cast(tf.image.resize(img, (128,128)), tf.float32)
#     imageio.imwrite(resized_negative_save_path+x, resized_img)
#     print("img {} done".format(x))    
# print("******************** TEST NEG COMPLETED *********************************")


        
        

Lossy conversion from float32 to uint8. Range [1.6953125, 252.74609375]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 248.2587890625]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 222.90045166015625]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 214.0859375]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 249.17770385742188]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 254.40234375]. Conv

img IM-0031-0001.jpeg done
img IM-0025-0001.jpeg done
img NORMAL2-IM-0272-0001.jpeg done
img NORMAL2-IM-0102-0001.jpeg done
img NORMAL2-IM-0229-0001.jpeg done
img NORMAL2-IM-0315-0001.jpeg done
img NORMAL2-IM-0123-0001.jpeg done
img NORMAL2-IM-0301-0001.jpeg done
img NORMAL2-IM-0246-0001-0002.jpeg done
img IM-0010-0001.jpeg done
img IM-0046-0001.jpeg done


Lossy conversion from float32 to uint8. Range [0.0, 253.1484375]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 254.25]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 254.73046875]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 253.1875]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [2.7215576171875, 253.333740234375]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 254.70703125]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [2.212890625, 254.8125]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 249.109375]. Co

img NORMAL2-IM-0357-0001.jpeg done
img NORMAL2-IM-0027-0001.jpeg done
img NORMAL2-IM-0033-0001.jpeg done
img NORMAL2-IM-0343-0001.jpeg done
img IM-0028-0001.jpeg done
img NORMAL2-IM-0339-0001.jpeg done
img NORMAL2-IM-0173-0001-0002.jpeg done
img NORMAL2-IM-0012-0001.jpeg done
img NORMAL2-IM-0362-0001.jpeg done
img IM-0009-0001.jpeg done
img NORMAL2-IM-0297-0001.jpeg done
img IM-0073-0001.jpeg done
img IM-0067-0001.jpeg done


Lossy conversion from float32 to uint8. Range [0.0, 254.04443359375]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 254.984375]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 254.9677734375]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 253.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [1.1953125, 254.890625]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 254.9560546875]. Convert image to uint

...

img NORMAL2-IM-0030-0001.jpeg done
img IM-0103-0001.jpeg done
img IM-0045-0001.jpeg done
img NORMAL2-IM-0354-0001.jpeg done
img NORMAL2-IM-0041-0001.jpeg done
img NORMAL2-IM-0277-0001.jpeg done
img NORMAL2-IM-0107-0001.jpeg done
img NORMAL2-IM-0331-0001.jpeg done
img IM-0093-0001.jpeg done
img NORMAL2-IM-0219-0001.jpeg done
img IM-0087-0001.jpeg done
img NORMAL2-IM-0173-0001-0001.jpeg done
img NORMAL2-IM-0325-0001.jpeg done
img IM-0015-0001.jpeg done


Lossy conversion from float32 to uint8. Range [0.0, 254.96484375]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 249.31640625]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 254.328125]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 248.85838317871094]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 253.74383544921875]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [9.121231079101562, 254.9904327392

img NORMAL2-IM-0238-0001.jpeg done
img NORMAL2-IM-0132-0001.jpeg done
img NORMAL2-IM-0304-0001.jpeg done
img NORMAL2-IM-0256-0001.jpeg done
img NORMAL2-IM-0060-0001.jpeg done
img NORMAL2-IM-0310-0001.jpeg done
img NORMAL2-IM-0195-0001.jpeg done
img IM-0001-0001.jpeg done
img IM-0111-0001.jpeg done
img NORMAL2-IM-0346-0001.jpeg done
img NORMAL2-IM-0352-0001.jpeg done
img IM-0039-0001.jpeg done
img NORMAL2-IM-0058-0001.jpeg done
img NORMAL2-IM-0328-0001.jpeg done


Lossy conversion from float32 to uint8. Range [1.720458984375, 244.73779296875]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 253.7734375]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 203.50772094726562]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 232.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 254.1890869140625]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 210.26806640625]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 254.9644775390625]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0,

img IM-0043-0001.jpeg done
img IM-0105-0001.jpeg done
img NORMAL2-IM-0373-0001.jpeg done
img NORMAL2-IM-0145-0001.jpeg done
img NORMAL2-IM-0286-0001.jpeg done
img NORMAL2-IM-0079-0001.jpeg done
img NORMAL2-IM-0309-0001.jpeg done
img NORMAL2-IM-0246-0001-0001.jpeg done
img NORMAL2-IM-0292-0001.jpeg done
img NORMAL2-IM-0221-0001.jpeg done
img NORMAL2-IM-0198-0001.jpeg done
******************** TEST NEG COMPLETED *********************************


## Loading, resizing, classifying images

In [290]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
          
def returnLabel(path):
    imgClass = tf.strings.split(path,'/')[-2]
    return 0 if imgClass == 'NORMAL' else 1

def returnImg(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    imgClass = returnLabel(path)
    return tf.image.resize(img, (256,256)), imgClass

def getX(image, label):
    return image

def getY(images, label):
    return label


In [291]:
train_path = 'Data/chest_xray/train'
list_ds = tf.data.Dataset.list_files(str(train_path+'/*/*'))
labeled_ds = list_ds.map(returnImg, num_parallel_calls=AUTOTUNE)
labeled_ds

<ParallelMapDataset shapes: ((256, 256, 3), ()), types: (tf.float32, tf.int32)>

In [298]:
X_train = labeled_ds.map(getX)
Y_train = labeled_ds.map(getY)

## Model


In [309]:
for x in X_train.take(10):
    print(x)

tf.Tensor(
[[[0.28440237 0.28440237 0.28440237]
  [0.46291313 0.46291313 0.46291313]
  [0.5491351  0.5491351  0.5491351 ]
  ...
  [0.         0.         0.        ]
  [0.01336278 0.01336278 0.01336278]
  [0.08753829 0.08753829 0.08753829]]

 [[0.26599264 0.26599264 0.26599264]
  [0.3344659  0.3344659  0.3344659 ]
  [0.49562657 0.49562657 0.49562657]
  ...
  [0.         0.         0.        ]
  [0.00997243 0.00997243 0.00997243]
  [0.08541173 0.08541173 0.08541173]]

 [[0.2657311  0.2657311  0.2657311 ]
  [0.27040273 0.27040273 0.27040273]
  [0.41483328 0.41483328 0.41483328]
  ...
  [0.         0.         0.        ]
  [0.01602329 0.01602329 0.01602329]
  [0.08821232 0.08821232 0.08821232]]

 ...

 [[0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  ...
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]]

 [[0.         0.         0.        ]
  [0.         0.         0